In [37]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence

from keras.preprocessing.text import Tokenizer
from keras.models import load_model

In [38]:
filenames = ["reviews_Home_and_Kitchen_5.json"]
filename = "reviews_Musical_Instruments_5.json"
t = Tokenizer()
max_features = 20000
maxlen = 100
batch_size = 32

dictionary_size = 0
embed_dim = 300
num_hidden_unit_lstm = 300
num_filter = 100
window_size = 3
pool_size = 2
batch_size = 10
learning_rate = 0.1
dropout_embed = 0.5
dropout_blstm = 0.2
dropout_penultimate = 0.4
l2_lambda = 0.000001

randomShuffle = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)

In [39]:
def getXY(filename, num_class):
    data = pd.read_json(filename, lines=True, orient="frame")
    data['overall'] = np.where(data['overall'] < 3, 0, data.overall)
    data['overall'] = np.where(data['overall'] > 3, 1, data.overall)
    
    # down sample
    data_0 = data.loc[data['overall'] == 0]
    data_1 = data.loc[data['overall'] == 1].sample(len(data_0))
    
    data = data_0.append(data_1).sample(frac=1)
    
    good_columns = [3]
    onehot_encoder = OneHotEncoder(sparse=False)
    data_X = data.iloc[:, good_columns]
    data_Y = data.iloc[:, 2]
    
    reviewTextList = data.reviewText.values
    t.fit_on_texts(reviewTextList)
    text = t.texts_to_matrix(reviewTextList, mode='count')
    
    
    data_X = text
    data_Y = np_utils.to_categorical(data_Y, num_class)
    return data_X, data_Y, text.shape[1]

In [40]:
def SVMModel(num_class, input_dim, x_train, x_test, y_train, y_test, batch_size, epochs, name_model):
    print("x_train: ", x_train.shape)
    print("y_train: ", y_train.shape)
    print("x_test: ", x_test.shape)
    print("y_test: ", y_test.shape)
#     print("x_train:",x_train[0])
#     print("x_test:",x_test[0])
#     print('y_train:',y_train[0])
#     print('y_test',y_test[0])
    model = Sequential()
    model.add(Dense(2,input_dim=input_dim,activation='softmax'))
    print(model.summary())
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    history=model.fit(x_train,y_train,
                      batch_size=80,
                      epochs=100,
                      validation_data=(x_test,y_test),
                      )
    score=model.evaluate(x_test,y_test)
    print('Test loss:',score[0])
    print('Test accuracy:',score[1])
    plt.figure('categorical_accuracy')
    plt.xlabel('Iterations')
    plt.ylabel('Categorical Accuracy')
    plt.plot(history.history['categorical_accuracy'])
    plt.show()
    y_test=np.argmax(y_test,axis=1)
    y_pred=model.predict_classes(x_test)
    print(classification_report(y_test,y_pred))
    
    model.save(name_model)
    del model

In [42]:
num_class = 2
X, Y, input_dim = getXY(filename, num_class)

In [45]:
for filename in filenames:
    print("-----------------------------")
    print(filename + ":")
    X, Y, input_dim = getXY(filename, num_class)
    print("input dimension: ", input_dim)
    for train_index, test_index in randomShuffle.split(X):
        X_train = X[train_index]
        Y_train = Y[train_index, :]
        X_test = X[test_index]
        Y_test = Y[test_index, :]
        
        bidirectionalLSTMCNNModel(num_class, input_dim, X_train, X_test, Y_train, Y_test, 10, 100, 'SVM.H5')

-----------------------------
reviews_Home_and_Kitchen_5.json:


MemoryError: 

In [ ]:
model = load_model('SVM.h5')